# 006: Operation Graphs - Academic Claim Validation

Building on ReAct patterns from tutorial 005, this demonstrates sequential coordination using Operation Graphs to orchestrate ReaderTool workflows for academic claim validation.

## What You'll Learn

1. **Sequential Coordination**: Building context step-by-step through dependent operations
2. **ReaderTool Integration**: Document chunking and progressive reading strategies  
3. **Structured Extraction**: Using Pydantic models for reliable claim extraction
4. **Operation Dependencies**: How operations build on previous results

## Use Case: Validating a Theoretical Framework Paper

We'll validate claims in an academic paper about capability-based security by:
- Reading document chunks progressively with ReaderTool
- Building understanding through sequential analysis
- Extracting verifiable claims with structured output formats
- Demonstrating how each operation builds on the previous one's results

In [1]:
# Setup and imports
from typing import Literal
from pathlib import Path
from pydantic import BaseModel, Field

from lionagi import Branch, Session, Builder, types, iModel
from lionagi.tools.types import ReaderTool

# Target document - complex theoretical framework
here = Path().cwd()
document_path = here / "data" / "006_lion_proof_ch2.md"

print("✅ Environment setup complete")
print(f"📄 Target: {document_path.name}")
print("🎯 Goal: Validate academic claims using coordinated ReAct workflows")

✅ Environment setup complete
📄 Target: 006_lion_proof_ch2.md
🎯 Goal: Validate academic claims using coordinated ReAct workflows


In [2]:
# Data models for structured responses
class Claim(BaseModel):
    claim: str
    type: Literal["citation", "performance", "technical", "other"]
    location: str = Field(..., description="Section/paragraph reference")
    verifiability: Literal["high", "medium", "low"]
    search_strategy: str = Field(..., description="How to verify this claim")


class ClaimExtraction(BaseModel):
    claims: list[Claim]


print("✅ Data models defined")

✅ Data models defined


## Pattern 1: Sequential Document Analysis

Build understanding step-by-step: Open → Analyze → Extract claims

In [3]:
async def sequential_analysis():
    """Sequential workflow: open → analyze structure → extract claims."""

    # Create branch with ReaderTool
    branch = Branch(
        tools=[ReaderTool], chat_model=iModel(model="openai/gpt-4.1-mini")
    )
    session = Session(default_branch=branch)
    builder = Builder()

    # Step 1: Open and understand document
    doc_reader = builder.add_operation(
        "ReAct",
        node_id="open_document",
        instruct=types.Instruct(
            instruction="Use ReaderTool to open and analyze the theoretical framework document. Understand its structure and identify sections containing verifiable claims.",
            context={"document_path": str(document_path)},
        ),
        tools=["reader_tool"],
        max_extensions=2,
        verbose=True,
        verbose_length=200,
    )

    # Step 2: Progressive content analysis
    content_analyzer = builder.add_operation(
        "ReAct",
        node_id="analyze_content",
        depends_on=[doc_reader],
        instruct=types.Instruct(
            instruction="Read through key sections to identify citations, technical claims, and performance metrics that can be verified."
        ),
        response_format=types.Outline,
        tools=["reader_tool"],
        max_extensions=3,
        verbose=True,
        verbose_length=200,
    )

    # Step 3: Extract specific claims
    claim_extractor = builder.add_operation(
        "ReAct",
        node_id="extract_claims",
        depends_on=[content_analyzer],
        instruct=types.Instruct(
            instruction="Extract 5-7 specific, verifiable claims. Prioritize citations, performance metrics, and technical assertions."
        ),
        response_format=ClaimExtraction,
        tools=["reader_tool"],
        max_extensions=3,
        verbose=True,
        verbose_length=200,
    )

    # Execute workflow
    graph = builder.get_graph()
    print("🔗 Executing sequential analysis...")

    result = await session.flow(graph, parallel=False, verbose=True)

    return result


# Execute sequential analysis
result = await sequential_analysis()

🔗 Executing sequential analysis...
Operation 4ced7250 waiting for 8482441d
Operation 22a5974b waiting for 4ced7250
Executing operation: 8482441d
Invoking action reader_tool with {'action': 'open', 'path_or_url': '/Users/lion/lio....
Action reader_tool invoked, status: completed.


### ReAct Round No.1 Analysis:
```yaml
analysis: I will first open the theoretical framework document located at /Users/lion/lionagi/cookbooks/data/006_lion_proof_ch2.md to analyze its structure. After opening it, I will read the i...

[Truncated output]
```
---------

Invoking action reader_tool with {'action': 'read', 'doc_id': 'DOC_3654073707073659....
Action reader_tool invoked, status: completed.


### ReAct Round No.2 Analysis:
```yaml
analysis: The document /Users/lion/lionagi/cookbooks/data/006_lion_proof_ch2.md has been successfully opened and contains 52,767 characters (~12,127 tokens). To understand the theoretical fram...

[Truncated output]
```
---------

Invoking action reader_tool with {'action': 'read', 'doc_id': 'DOC_3654073707073659....
Action reader_tool invoked, status: completed.


...

### ReAct Final Answer:
```yaml
topic: Formal Security Guarantees in Capability Systems
items:
  - heading: Introduction and Motivation
    summary: Overview of the legitimacy of capability-based security and motivation for ...

[Truncated output]
```
---------

Completed operation: 4ced7250
Operation 22a5974b starting with fresh context
Executing operation: 22a5974b


### ReAct Round No.1 Analysis:
```yaml
analysis: 1. The core verifiable claim is from Theorem 2.1, asserting that capability authority is preserved during inter-component message passing, formalized by the cryptographic unforgeabil...

[Truncated output]
```
---------

### ReAct Final Answer:
```yaml
claims:
  - claim: Theorem 2.1 asserts that the capability authority is preserved during cross-component message passing, formalized by cryptographic unforgeability and authority invariance of...

[Truncated output]
```
---------

Completed operation: 22a5974b


In [4]:
from IPython.display import display, Markdown

# Display results
for node_id, data in result["operation_results"].items():
    if isinstance(data, types.Outline):
        md_content = f"""
## 📄 Document Structure ({node_id})

**Topic:** {data.topic}

### Key Sections:
"""
        for item in data.items[:3]:  # Show first 3
            md_content += f"- **{item.heading}**: {item.summary}\n"

        display(Markdown(md_content))

    elif isinstance(data, ClaimExtraction):
        md_content = f"""
## 📑 Extracted Claims ({node_id})

Found **{len(data.claims)}** verifiable claims:

"""
        for i, claim in enumerate(data.claims, 1):
            md_content += f"""
### {i}. [{claim.type.upper()}] {claim.claim}

- **Location:** {claim.location}  
- **Verifiability:** {claim.verifiability}
- **Search Strategy:** {claim.search_strategy}

"""
        display(Markdown(md_content))

display(Markdown("## ✅ Sequential analysis completed"))


## 📄 Document Structure (4ced7250-99e5-4819-bc06-c48ea1857998)

**Topic:** Formal Security Guarantees in Capability Systems

### Key Sections:
- **Introduction and Motivation**: Overview of the legitimacy of capability-based security and motivation for formal proofs.
- **Formal System Models and Definitions**: Precise formal definitions of capabilities, system components, and security invariants.
- **Main Theorems and Proofs**: Theorems 2.1 to 2.4 detailing security preservation, composition, and enforcement, supported by lemmas and mechanized proofs.



## 📑 Extracted Claims (22a5974b-7642-41f7-8728-398c35273c07)

Found **7** verifiable claims:


### 1. [TECHNICAL] Theorem 2.1 asserts that the capability authority is preserved during cross-component message passing, formalized by cryptographic unforgeability and authority invariance of capabilities.

- **Location:** Section 2.3  
- **Verifiability:** high
- **Search Strategy:** Review formal definitions of capabilities and proof of Theorem 2.1.


### 2. [TECHNICAL] Theorem 2.2 states that security properties remain intact when components are composed, supported by lemmas about unforgeability, authority confinement, and policy compliance.

- **Location:** Section 2.4  
- **Verifiability:** high
- **Search Strategy:** Examine lemmas supporting the preservation of security invariants during composition.


### 3. [TECHNICAL] Theorem 2.3 guarantees Lion's capability system prevents confused deputy attacks by enforcing explicit capability transfer and restricting ambient authority.

- **Location:** Section 2.5  
- **Verifiability:** high
- **Search Strategy:** Analyze lemmas on the absence of ambient authority and explicit delegation protocols.


### 4. [TECHNICAL] Theorem 2.4 ensures the automatic enforcement of the principle of least authority, achieved via static type checks and runtime capability derivation, as supported by formal lemmas.

- **Location:** Section 2.6  
- **Verifiability:** high
- **Search Strategy:** Review lemmas on type enforcement, capability derivation, and minimal scope capabilities.


### 5. [CITATION] Mechanized proofs in TLA+ and Lean verify core invariants such as unforgeability, authority preservation, and proper attenuation, providing formal validation of the theoretical claims.

- **Location:** Section 2.8  
- **Verifiability:** high
- **Search Strategy:** Verify the mechanized proof artifacts and their correspondence to the stated theorems.


### 6. [TECHNICAL] Cryptographic signatures and formal definitions of capabilities serve as verifiable assertions about unforgeability and authority invariance during capability transfers.

- **Location:** Throughout the formal definitions and transfer protocols  
- **Verifiability:** medium
- **Search Strategy:** Review cryptographic protocol details and formal definitions in the document.


### 7. [PERFORMANCE] The formal security invariants hold across modeled actions such as message passing, component composition, capability delegation, and runtime attenuation, as confirmed by model checking and mechanized proofs.

- **Location:** Sections 2.8 and Appendix A.2  
- **Verifiability:** high
- **Search Strategy:** Examine model checking results and mechanized proof summaries for invariant validation.



## ✅ Sequential analysis completed